#MA707 Preprocessing Fixes

In [2]:
%run "../Report/0.2 Feature creation (inc)"

In [3]:
%run "/Courses/MA707/Groups/Blackjack/Report - Final/0.2 Feature creation (inc)"

In [4]:
%sh ls /dbfs/mnt/group-ma707/data/*

/dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv
/dbfs/mnt/group-ma707/data/mining_com_coal.csv
/dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv

In [5]:
%python
import pandas as pd
bci_pdf = pd.read_csv('/dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv') \
            .rename(columns={'P3A~IV':'P3A_IV'}) \
            .assign(date=lambda pdf: pd.to_datetime(pdf.Date)) \
            .drop('Date', axis=1) \
            .sort_index(ascending=True)
bci_pdf.columns = bci_pdf.columns.str.lower()
bci_pdf.info() # 1602 non-null for all vars same count for index

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1602 entries, 0 to 1601
Data columns (total 26 columns):
bci 1602 non-null int64
c5 1602 non-null float64
c7 1602 non-null float64
p1a_03 1602 non-null int64
p2a_03 1602 non-null int64
p4_03 1602 non-null int64
p3a_iv 1602 non-null float64
shfe_al3 1602 non-null float64
rici 1602 non-null float64
ice_kc3 1602 non-null float64
cme_sm3 1602 non-null float64
cme_lc2 1602 non-null float64
opec_orb 1602 non-null float64
shfe_cu3 1602 non-null float64
cme_ln1 1602 non-null float64
cme_fc3 1602 non-null float64
p3a_03 1602 non-null int64
shfe_rb3 1602 non-null int64
cme_s2 1602 non-null float64
ice_sb3 1602 non-null float64
cme_ln3 1602 non-null float64
cme_ln2 1602 non-null float64
ice_tib3 1602 non-null float64
ice_tib4 1602 non-null float64
bci_5tc 1602 non-null int64
date 1602 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(18), int64(7)
memory usage: 325.5 KB

In [6]:
%python
import numpy as np
import pandas as pd
coal_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', 
            encoding='ISO-8859-1'
           ) \
  .loc[:,['date','tags','title','content']] \
  .fillna({'tags'   :'',
           'content':'',
           'title'  :''
          }) \
  .assign(date   =lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date).dt.date)) \
  .groupby(by='date') \
  .agg({'tags'   : lambda ser: ' '.join(ser),
        'content': lambda ser: ' '.join(ser),
        'title'  : lambda ser: ' '.join(ser)}) \
  .resample('D') \
  .pad() \
  .reset_index()
coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3943 entries, 0 to 3942
Data columns (total 4 columns):
date 3943 non-null datetime64[ns]
title 3943 non-null object
content 3943 non-null object
tags 3943 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 123.3+ KB

In [7]:
%python
import numpy as np
import pandas as pd
ore_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv', 
            encoding='ISO-8859-1'
           ) \
  .loc[:,['date','tags','title','content']] \
  .fillna({'tags'   :'',
           'content':'',
           'title'  :''
          }) \
  .assign(date = lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date,utc=True).dt.normalize().dt.date)) \
  .groupby(by='date') \
  .agg({'tags'   : lambda ser: ' '.join(ser),
        'content': lambda ser: ' '.join(ser),
        'title'  : lambda ser: ' '.join(ser)}) \
  .resample('D') \
  .pad() \
  .reset_index()
ore_pdf.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3928 entries, 0 to 3927
Data columns (total 4 columns):
date 3928 non-null datetime64[ns]
title 3928 non-null object
content 3928 non-null object
tags 3928 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 122.8+ KB

In [8]:
ore_pdf.columns=['date','title_ore','content_ore','tags_ore']

In [9]:
%python
import pandas as pd
bci_coal_pdf = \
pd.concat(objs=[ bci_pdf.set_index('date'), 
                coal_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 29 columns):
date 1598 non-null datetime64[ns]
bci 1598 non-null int64
c5 1598 non-null float64
c7 1598 non-null float64
p1a_03 1598 non-null int64
p2a_03 1598 non-null int64
p4_03 1598 non-null int64
p3a_iv 1598 non-null float64
shfe_al3 1598 non-null float64
rici 1598 non-null float64
ice_kc3 1598 non-null float64
cme_sm3 1598 non-null float64
cme_lc2 1598 non-null float64
opec_orb 1598 non-null float64
shfe_cu3 1598 non-null float64
cme_ln1 1598 non-null float64
cme_fc3 1598 non-null float64
p3a_03 1598 non-null int64
shfe_rb3 1598 non-null int64
cme_s2 1598 non-null float64
ice_sb3 1598 non-null float64
cme_ln3 1598 non-null float64
cme_ln2 1598 non-null float64
ice_tib3 1598 non-null float64
ice_tib4 1598 non-null float64
bci_5tc 1598 non-null int64
title 1598 non-null object
content 1598 non-null object
tags 1598 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(3)
memory usage: 362.1+ KB

In [10]:
%python
import pandas as pd
bci_ironore_pdf = \
pd.concat(objs=[bci_pdf.set_index('date'), 
                ore_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_ironore_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 29 columns):
date 1598 non-null datetime64[ns]
bci 1598 non-null int64
c5 1598 non-null float64
c7 1598 non-null float64
p1a_03 1598 non-null int64
p2a_03 1598 non-null int64
p4_03 1598 non-null int64
p3a_iv 1598 non-null float64
shfe_al3 1598 non-null float64
rici 1598 non-null float64
ice_kc3 1598 non-null float64
cme_sm3 1598 non-null float64
cme_lc2 1598 non-null float64
opec_orb 1598 non-null float64
shfe_cu3 1598 non-null float64
cme_ln1 1598 non-null float64
cme_fc3 1598 non-null float64
p3a_03 1598 non-null int64
shfe_rb3 1598 non-null int64
cme_s2 1598 non-null float64
ice_sb3 1598 non-null float64
cme_ln3 1598 non-null float64
cme_ln2 1598 non-null float64
ice_tib3 1598 non-null float64
ice_tib4 1598 non-null float64
bci_5tc 1598 non-null int64
title_ore 1598 non-null object
content_ore 1598 non-null object
tags_ore 1598 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(3)
memory usage: 362.1+ KB

In [11]:
%python
import pandas as pd
bci_dual_pdf = \
pd.concat(objs=[bci_coal_pdf.set_index('date'), 
                ore_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_dual_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 32 columns):
date 1598 non-null datetime64[ns]
bci 1598 non-null int64
c5 1598 non-null float64
c7 1598 non-null float64
p1a_03 1598 non-null int64
p2a_03 1598 non-null int64
p4_03 1598 non-null int64
p3a_iv 1598 non-null float64
shfe_al3 1598 non-null float64
rici 1598 non-null float64
ice_kc3 1598 non-null float64
cme_sm3 1598 non-null float64
cme_lc2 1598 non-null float64
opec_orb 1598 non-null float64
shfe_cu3 1598 non-null float64
cme_ln1 1598 non-null float64
cme_fc3 1598 non-null float64
p3a_03 1598 non-null int64
shfe_rb3 1598 non-null int64
cme_s2 1598 non-null float64
ice_sb3 1598 non-null float64
cme_ln3 1598 non-null float64
cme_ln2 1598 non-null float64
ice_tib3 1598 non-null float64
ice_tib4 1598 non-null float64
bci_5tc 1598 non-null int64
title 1598 non-null object
content 1598 non-null object
tags 1598 non-null object
title_ore 1598 non-null object
content_ore 1598 non-null object
tags_ore 1598 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(6)
memory usage: 399.6+ KB

In [12]:
%python
import pandas as pd
bci_dual_pdf = \
pd.concat(objs=[ bci_pdf.set_index('date'), 
                 ore_pdf.set_index('date'),
                coal_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_dual_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 32 columns):
date 1598 non-null datetime64[ns]
bci 1598 non-null int64
c5 1598 non-null float64
c7 1598 non-null float64
p1a_03 1598 non-null int64
p2a_03 1598 non-null int64
p4_03 1598 non-null int64
p3a_iv 1598 non-null float64
shfe_al3 1598 non-null float64
rici 1598 non-null float64
ice_kc3 1598 non-null float64
cme_sm3 1598 non-null float64
cme_lc2 1598 non-null float64
opec_orb 1598 non-null float64
shfe_cu3 1598 non-null float64
cme_ln1 1598 non-null float64
cme_fc3 1598 non-null float64
p3a_03 1598 non-null int64
shfe_rb3 1598 non-null int64
cme_s2 1598 non-null float64
ice_sb3 1598 non-null float64
cme_ln3 1598 non-null float64
cme_ln2 1598 non-null float64
ice_tib3 1598 non-null float64
ice_tib4 1598 non-null float64
bci_5tc 1598 non-null int64
title_ore 1598 non-null object
content_ore 1598 non-null object
tags_ore 1598 non-null object
title 1598 non-null object
content 1598 non-null object
tags 1598 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(6)
memory usage: 399.6+ KB

In [14]:
%python 
def get_count_vect_all_three_plus_all_ts_pipe():
  from sklearn.pipeline import FeatureUnion, Pipeline
  return Pipeline(steps=[
    ('fea_one', FeatureUnionDF(transformer_list=[
      ('tgt_var'    ,CreateTargetVarDF(var='bci_5tc')),
      ('dt_vars'    ,CreateDatetimeVarsDF(var='date')),
      ('lag_txt_vars',CreateLagVarsDF(var_list=['tags','content','title'],
                                      lag_list=[3])),
    ])),
    ('drop_na_rows'  ,DropNaRowsDF(how='any')),
    ('fea_two', FeatureUnionDF(transformer_list=[
      ('named_vars' ,CreateNamedVarsDF(except_list=['tags_lag3','content_lag3','title_lag3'])),
      ('cnt_tags'   , CountVectColDF(col_name=   'tags_lag3',prefix='cnt_tags_'   ,add_stop_words=[])),
      ('cnt_content', CountVectColDF(col_name='content_lag3',prefix='cnt_content_',add_stop_words=[])),  
      ('cnt_title'  , CountVectColDF(col_name=  'title_lag3',prefix='cnt_title_'  ,add_stop_words=[])),  
    ])),
    ('drop_na_rows_again'  ,DropNaRowsDF(how='any')),
  ])

Ask about this error - creating new feature target pipelines is simple once we determine why this isnt working - because its just about changing what we count, and what we look at in terms of lagged variables, tfidf vectorizer, etc.

In [16]:
get_count_vect_all_three_plus_all_ts_pipe() \
  .fit(bci_coal_pdf) \
  .transform(bci_coal_pdf)\
  .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1592 entries, 3 to 1594
Columns: 41462 entries, year to cnt_title_zuckerberg
dtypes: float64(41462)
memory usage: 503.6 MB